## Configuration 

At the time of writing this notebook, you will need to install LlamaIndex version `0.8.0`.

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.Client(
    embedded_options=weaviate.embedded.EmbeddedOptions()
)

## Create Schema

In [ ]:
schema = {
   "classes": [
       {
           "class": "BlogPost",
           "description": "Blog post from the Weaviate website.",
           "vectorizer": "text2vec-openai",
           "moduleConfig": {
               "generative-openai": { 
                    "model": "gpt-3.5-turbo"
                }
           },
           "properties": [
               {
                  "name": "Content",
                  "dataType": ["text"],
                  "description": "Content from the blog post",
               }
            ]
        }
    ]
}

client.schema.delete_all()

client.schema.create(schema)

print("Schema was created.")

## Add Data

In [ ]:
from llama_index import SimpleDirectoryReader

blogs = SimpleDirectoryReader('./data').load_data()

## Setup Weaviate Vector Store

In [ ]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex
from llama_index.storage.storage_context import StorageContext
import os

openai_api_key = os.environ["OPENAI_API_KEY"]

# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="BlogPost", text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex.from_documents(blogs, storage_context = storage_context)

## Query without Self-Corrrecting

In [ ]:
base_query_engine = index.as_query_engine()
query = "What is Ref2Vec?"

response = base_query_engine.query(query)
print(response)

## Configure Self-Correcting Query Engine

In [ ]:
from llama_index.evaluation.guideline_eval import GuidelineEvaluator, DEFAULT_GUIDELINES
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.query_engine.retry_query_engine import (
    RetryGuidelineQueryEngine,
)

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES + "\nThe response should try to summarize where possible.\n"
    "The response should mention Weaviate and not be too vauge.\n"
)

In [ ]:
typed_response = response if isinstance(response, Response) else response.get_response()
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

In [ ]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    base_query_engine, guideline_eval, resynthesize_query=True
)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(retry_guideline_response)